In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Using TensorFlow backend.


In [2]:
from zipfile import ZipFile
file_name = "data/train.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [3]:
df = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')
df.head()

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [64]:
TRAIN_DIR = 'train'

IMG_SIZE=24
data=[]
labels_240=[]
i = 0
for img in tqdm(os.listdir(TRAIN_DIR)):
    path = os.path.join(TRAIN_DIR,img)
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE,IMG_SIZE))
    image=img_to_array(img_data)
    data.append(image)


100%|██████████| 60000/60000 [00:12<00:00, 4796.73it/s]


In [66]:
image.shape

(24, 24, 3)

In [67]:
labels_240=[]
for img in tqdm(os.listdir(TRAIN_DIR)):
    number = int(img[:-4])  
    categ_num = int(df[df.id==number].label)
    labels_240.append(categ_num)

100%|██████████| 60000/60000 [00:59<00:00, 1010.67it/s]


In [68]:
num_classes=len(np.unique(labels_240))
data=np.array(data,dtype="float32")/255.0
labels_240=np.array(labels_240)
lb=LabelBinarizer()
labels_240=lb.fit_transform(labels_240)

In [69]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [70]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="linear",input_shape=(24,24,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10
                ,activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [71]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(data,labels_240,test_size=0.25,random_state=47)

In [72]:

x_train = np.array([i[0] for i in x_train]).reshape(-1,240,240,3)
x_test = np.array([i[0] for i in x_test]).reshape(-1,240,240,3)

ValueError: cannot reshape array of size 3240000 into shape (240,240,3)

In [73]:
  len(x_train)
  len(y_train)
  len(y_test)
  len(x_test)
  print('')
  len(x_train[0])

45000

45000

15000

15000

24

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 24, 24, 32)        896       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 24, 24, 32)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 8, 8, 32)          0         
__________

In [75]:
aug=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,horizontal_flip=True,fill_mode="nearest")


In [78]:
checkpoint=ModelCheckpoint(filepath='data/model_best2.hdf5',
                           save_best_only=True,verbose=1)


In [80]:
train=model.fit_generator(aug.flow(x_train,y_train,batch_size=64),validation_data=(x_test,y_test),
                          steps_per_epoch=len(x_train)/64,epochs=1,verbose=1,callbacks=[checkpoint])

Epoch 1/1
704/703 [==============================] - 62s 89ms/step - loss: 0.6914 - acc: 0.7384 - val_loss: 0.5404 - val_acc: 0.7860

Epoch 00001: val_loss improved from 0.55968 to 0.54038, saving model to data/model_best2.hdf5
